# <b>Image features demo 1</b>

The AIServiceVisionClient offers the <b>image classification</b> feature. This notebook aims to provide overall clarity about the feature to the user in terms of requirements, usage and the output of the API.<br>
<ul>
    <li>The raw output is saved as <code>response_image_demo1.json</code> in the <code>output</code> directory. </li>
    <li>Image classes are printed in descending order of their confidence levels in the last cell of the notebook.</li>
</ul>

### Steps to run the notebook:
<details>
    <summary>Notebook session setup</summary>
    <ol>
        <li><font size="2">Installing the OCI Vision SDK</font></li>
        <li><font size="2">Installing other dependencies</font></li>
        <li><font size="2">Setup sample input images</font></li>
        <li><font size="2">Setup helper .py files</font></li>
        <li><font size="2">Create output folder</font></li>
    </ol>
</details>

<details>
    <summary>Importing the required modules</summary>
</details>

<details>
    <summary>Setting the input variables</summary>
     <font size="2">The user can change the input variables, if necessary. They have been assigned default values.</font>
</details>

<details>
    <summary>Running the main pipeline</summary>
    <font size="2">Run all cells to get the output in the <code>output</code> directory. </font><br>
</details>

### Notebook session setup
<details>
    <summary>Instructions</summary>
    <ul>
        <li><font size="2">The user needs to setup only once.</font></li>
        <li><font size="2">Uncomment the commented cells and run once to setup.</font></li>
        <li><font size="2">Comment back the same cells to avoid running again.</font></li>
    </ul>
</details>

#### Installing the OCI Python SDK

In [ ]:
# !pip install oci==2.57.0

#### Installing other dependencies

In [ ]:
# !pip install matplotlib==3.3.4
# !pip install pandas==1.1.5

#### Setup sample input images

In [ ]:
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/car.jpg"
# !mkdir data
# !mv car.jpg data

#### Setup helper .py files

In [ ]:
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/output_plot.py"
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/analyze_image_utils.py"
# !mkdir helper
# !mv output_plot.py helper
# !mv analyze_image_utils.py helper

#### Create output folder

In [ ]:
# !mkdir output

### Imports

In [ ]:
import base64
import io
import json
from PIL import Image
import matplotlib.pyplot as plt
import requests
import oci
from helper.analyze_image_utils import is_url, display_classes, clean_output

### Set input variables
<details>
<summary><font size="3">input_path</font></summary>
<font size="2">The user can provide the image URL or filepath from the notebook session</font><br>
</details>
<details>
<summary><font size="3">max_results</font></summary>
    <font size="2">Provide the maximum number of results needed for image classification. This is an upper limit over the output classes, the API may detect lesser classes according to the image.</font><br>
</details>

In [ ]:
input_path = "data/car.jpg"
max_results = 5

### Authorize user config

In [ ]:
config = oci.config.from_file('~/.oci/config')

### View input image

In [ ]:
if is_url(input_path):
    encoded_string = base64.b64encode(requests.get(input_path).content)
else:
    with open(input_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())

image_data = base64.b64decode(encoded_string)
image = Image.open(io.BytesIO(image_data))
plt.gcf().set_dpi(200)
plt.axis('off')
plt.imshow(image)

### Create AI service vision client and get response object

In [ ]:
ai_service_vision_client = oci.ai_vision.AIServiceVisionClient(config=config)
analyze_image_details = oci.ai_vision.models.AnalyzeImageDetails()
inline_image_details = oci.ai_vision.models.InlineImageDetails()

image_classification_feature = oci.ai_vision.models.ImageClassificationFeature()
image_classification_feature.max_results = max_results
features = [image_classification_feature]
inline_image_details.data = encoded_string.decode('utf-8')
analyze_image_details.image = inline_image_details
analyze_image_details.features = features

res = ai_service_vision_client.analyze_image(analyze_image_details=analyze_image_details)

### Clean and save the API response as json

In [ ]:
res_json = json.loads(repr(res.data))
clean_res = clean_output(res_json)
with open('output/response_image_demo1.json', 'w') as fp:
    json.dump(clean_res, fp)

### Display the classes with their confidence levels

In [ ]:
if 'labels' in clean_res:
    display_classes(clean_res['labels'])